## Naive Bayes Using the Text Field

Train Test Split

In [ ]:
Target feature = ""
X = complaints[['complaint_narrative']]
y = complaints['issue']

X_train, X_test, y_train, y_test = train_test_split( X, y, random_state = 321, stratify = y)

Count Vectorizer - THIS IS NOT GONNA WORK.  Look at michael's notes and TOMO's code . . .  need a sparse matrix and all that jazz 

In [ ]:
#Fill in the code to fit and transform a CountVectorizer (using all defaults) on the text column of X_train and X_test
vect = CountVectorizer()

#Fit 
X_train_vec = vect.fit_transform(X_train["complaint_narrative"])
X_test_vec = vect.transform(X_test["complaint_narrative"]) 

look at the vocabulary

In [ ]:
vect.vocabulary_